In [2]:
#Web scraping imports 
from bs4 import BeautifulSoup as bs 
from selenium import webdriver
import time 

#Random import 
import numpy as np

#DataFrame import
import pandas as pd

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [3]:
#Code ready to pulll listings of only the main page for 20 pages ----------------------------------------

#Dictionary for storage
listings_dict = {'Price (usd)' : [], "Address" : [], 'Rooms' : [], 'Bathrooms' : [], 'Area (sqrt)' : [], 'Link' : []}

#Path to drivers - need 2 browsers 1 for the main page listing and another for the individual links of each linsting.
# If this is not done and we do everything in the same browser Zillow detects the crawler. 
# Remember to unzip the file of the drivers and get the absolute path to the driver not the relative path.
#In the folder, copy the chrome driver and rename the copy as chromedriver2. Make sure browser is linked to chromedriver AND browser2 is linked to chromedriver2
browser = webdriver.Chrome("/Users/Sebastian/Documents/GitHub/Data Visualization Bootcamp/Sebastian Homework/Goup Project - 2/TFH-Project/Zillow Scrapper/chromedriver")
#browser2 = webdriver.Chrome("/Users/Sebastian/Documents/GitHub/Data Visualization Bootcamp/Sebastian Homework/TFH-Project2/Zillow Scrapper/chromedriver2")

#starting page
page = 1 

#url to scrape starting at page 1
url = "https://www.zillow.com/homes/for_sale/fore_lt/austin_rb/pmf,pf_pt/?searchQueryState=%7B%22pagination%22%3A%7B%7D%2C%22usersSearchTerm%22%3A%22austin%22%2C%22mapBounds%22%3A%7B%22west%22%3A-98.3748184592871%2C%22east%22%3A-97.4409805686621%2C%22south%22%3A29.767357829257207%2C%22north%22%3A30.58768177330594%7D%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A10221%2C%22regionType%22%3A6%7D%5D%2C%22isMapVisible%22%3Atrue%2C%22filterState%22%3A%7B%22sort%22%3A%7B%22value%22%3A%22globalrelevanceex%22%7D%2C%22auc%22%3A%7B%22value%22%3Afalse%7D%2C%22nc%22%3A%7B%22value%22%3Afalse%7D%2C%22fsbo%22%3A%7B%22value%22%3Afalse%7D%2C%22cmsn%22%3A%7B%22value%22%3Afalse%7D%2C%22fsba%22%3A%7B%22value%22%3Afalse%7D%7D%2C%22isListVisible%22%3Atrue%7D"

#Open url -- We only want to open the browser once, so this is run outside the loop.
browser.get(url)

#while loop to iterate until condition is meet
while True:

    #Bot handler. -- Gives us 5 minutes to solve the human test and continues the code after it sees class = list-card-info.
    try:
        WebDriverWait(browser, 60*5).until(EC.presence_of_element_located((By.CLASS_NAME, "list-card-info")))
    except:
        print("Wait Timed out")

    

    #Prints current page to scrape
    print(f"Current page: {page}")

    #Random lag generator - try to keep the code to be detected as a bot 
    lag = np.random.uniform(0, 1.5*60)
    time.sleep(lag)

    #save the html source of the main page.
    html = browser.page_source

    #Use bs4 to parse the html response.
    soup = bs(html, "html.parser")

    #Results for all the listings in the page.
    listings = soup.find_all('div', class_="list-card-info")

    #For loop to evaluate each listing per page
    for listing in listings:

        #page index counter
        count = 0

        #Defines each variable for each iteration
        price = None 
        address = None 
        rooms = None 
        bathrooms = None 
        area = None 
        link = None 
        # typeProperty = None 
        # yearBuilt = None 
        # parking = None 
        # lot = None 
        # saleHistory = None 
        # taxHistory = None

        #Try the main page.
  
            #Price of the property
        try: 
            price = listing.find('div', class_="list-card-price").text
        except:
            price = "N/A"

            #Address
        try: 
            address = listing.find('address', class_="list-card-addr").text
        except:
             address = 'N/A'   

            #Rooms  - some are listed as Studio, bds and bd
        try:
            rooms = listing.find_all('li')[0].text
        except:
            rooms = "N/A" 

            #Bathrooms
        try:
            bathrooms = listing.find_all('li')[1].text
        except:
            bathrooms = "N/A"
            #Area of the house.
        try:  
            area = listing.find_all('li')[2].text.replace(" sqft","")
        except: 
            area = "N/A"
            #link of the individual listing.
        try:
            link = listing.a['href']
        except:
             link = "N/A"   

        #Only appends listings with complete information
        if (price and address and rooms and bathrooms and area and link):

            listingInformation = [price, address, rooms, bathrooms, area, link]

            #For loop to append the scraped information.
            for key,info in zip(listings_dict.keys(), listingInformation):
                listings_dict[key].append(info)

        #Adds 1 to the count at the end of the listing scrape
        count += 1
            

    #Adds 1 to the page counter
    page += 1

    #Finds elements to go next page
    try:
        nextpage = browser.find_element_by_css_selector("a.Button-wpcbcc-0.lcWnHB.PaginationButton-si2hz6-0.cUjspl")
    except:
        browser.quit()
        #browser2.quit()
        print("Scape Done!!!!")
        break

    #Zillow only loads 20 page (500 listings) at  time.
    if page <= 1:
        #clicks next page.
        nextpage.click()
        #Random lag generator - try to keep the code to be detected as a bot 
        lag = np.random.uniform(0, 1.5*60)
        time.sleep(lag)

    else:
        #When the code is done, this code kills the driver.
        browser.quit()
        #browser2.quit()
        print("Scape Done!!!!")
        break

Current page: 1
Scape Done!!!!


In [4]:
listings_df = pd.DataFrame(listings_dict)

listings_df.to_csv('zillowOuput.csv')

listings_df

,Price (usd),Address,Rooms,Bathrooms,Area (sqrt),Link
0,"$232,800","4704 Manchaca Rd, Austin, TX 78745",3 bds,2 ba,"1,657",https://www.zillow.com/homedetails/4704-Mancha...
1,"Est. $1,087,650","806 Bouldin Ave, Austin, TX 78704",4 bds,3 ba,"2,564",https://www.zillow.com/homedetails/806-Bouldin...
2,"Est. $438,311","2013 Shiloh Dr, Austin, TX 78745",-- bds,-- ba,"2,140",https://www.zillow.com/homedetails/2013-Shiloh...
3,$--,"4609 Duval St, Austin, TX 78751",1 bd,1 ba,513,https://www.zillow.com/homedetails/4609-Duval-...
4,"Est. $228,512","12412 Tay Ter, Austin, TX 78754",3 bds,2 ba,"1,203",https://www.zillow.com/homedetails/12412-Tay-T...
5,"Est. $540,083","5210 Joe Sayers Ave, Austin, TX 78756",1 bd,1 ba,850,https://www.zillow.com/homedetails/5210-Joe-Sa...
6,"Est. $175,865","604 Montopolis Dr, Austin, TX 78741",1 bd,1 ba,864,https://www.zillow.com/homedetails/604-Montopo...
7,"Est. $463,516","5503 Ameswood Dr, Austin, TX 78723",3 bds,2 ba,"2,120",https://www.zillow.com/homedetails/5503-Ameswo...
8,"Est. $565,378","5307 Harmon Ave, Austin, TX 78751",3 bds,2 ba,"1,551",https://www.zillow.com/homedetails/5307-Harmon...
9,"Est. $434,780","702 Zennia St, Austin, TX 78751",-- bds,-- ba,874,https://www.zillow.com/homedetails/702-Zennia-...


In [2]:
#Code ready to scrape main listings and individual listings for 20 pages of 1 city --------------------------------------------------------------------------------

#Dictionary for storage
listings_dict = {'Price (usd)' : [], "Address" : [], 'Rooms' : [], 'Bathrooms' : [], 'Area (sqrt)' : [], 'Link' : [], "typeProperty" : [], "yearBuilt" : [], "parking" : [], "lot" : [], "saleHistory" : [], "taxHistory" : [] }

#Path to drivers - need 2 browsers 1 for the main page listing and another for the individual links of each linsting.
# If this is not done and we do everything in the same browser Zillow detects the crawler. 
# Remember to unzip the file of the drivers and get the absolute path to the driver not the relative path.
#In the folder, copy the chrome driver and rename the copy as chromedriver2. Make sure browser is linked to chromedriver AND browser2 is linked to chromedriver2
browser = webdriver.Chrome("")
browser2 = webdriver.Chrome("")

#starting page
page = 1 

#url to scrape starting at page 1
url = "https://www.zillow.com/homes/Austin,-TX_rb/"

#Open url -- We only want to open the browser once, so this is run outside the loop.
browser.get(url)

#while loop to iterate until condition is meet
while True:

    #Bot handler. -- Gives us 5 minutes to solve the human test and continues the code after it sees class = list-card-info.
    try:
        WebDriverWait(browser, 60*5).until(EC.presence_of_element_located((By.CLASS_NAME, "list-card-info")))
    except:
        print("Wait Timed out")

    

    #Prints current page to scrape
    print(f"Current page: {page}")

    #Random lag generator - try to keep the code to be detected as a bot 
    lag = np.random.uniform(0, 2*60)
    time.sleep(lag)

    #save the html source of the main page.
    html = browser.page_source

    #Use bs4 to parse the html response.
    soup = bs(html, "html.parser")

    #Results for all the listings in the page.
    listings = soup.find_all('div', class_="list-card-info")

    #For loop to evaluate each listing per page
    for listing in listings:

        #page index counter
        count = 0

        #Defines each variable for each iteration
        price = None 
        address = None 
        rooms = None 
        bathrooms = None 
        area = None 
        link = None 
        typeProperty = None 
        yearBuilt = None 
        parking = None 
        lot = None 
        saleHistory = None 
        taxHistory = None

        #Try the main page.
        try:
            #Price of the property
            try:
                price = int(listing.find('div', class_="list-card-price").text.replace("$","").replace(",",""))
            except ValueError as e:
                print(f"{e} error at price at {count}")

            #Address
            try:
                address = listing.find('address', class_="list-card-addr").text
            except ValueError as e:
                print(f"{e} error at address at {count}")

            #Rooms  - some are listed as Studio, bds and bd
            try:
                try:
                    rooms = int(listing.find_all('li')[0].text.replace(" bds",""))
                except ValueError as e:
                    pass
                try:
                    rooms = int(listing.find_all('li')[0].text.replace("Studio","1"))
                except ValueError as e:
                    pass
                try:
                    rooms = int(listing.find_all('li')[0].text.replace(" bd",""))
                except ValueError as e:
                    pass
            except ValueError as e:
                print(f"{e} error at rooms at {count}")

            #Bathrooms
            try:
                bathrooms = int(listing.find_all('li')[1].text.replace(" ba",""))
            except ValueError:
                if (listing.find_all('li')[1].text == '-- ba'):
                    continue

            #Area of the house.
            try:
                area = int(listing.find_all('li')[2].text.replace(" sqft","").replace(",",""))
            except ValueError as e:
                print(f"{e} error at area at {count}")

            #link of the individual listing.
            try:
                link = listing.a['href']
            except ValueError as e:
                print(f"{e} error at rooms at {count}")

            #Opens the listing link using the second browser
            browser2.get(link)

            #Bot handler. -- Gives us 5 minutes to solve the human test and continues the code after it sees class = ds-body.
            try:
                WebDriverWait(browser2, 60*5).until(EC.presence_of_element_located((By.CLASS_NAME, "ds-body")))
            except:
                print("Wait Timed out")

            #Random lag generator - try to keep the code to be detected as a bot 
            lag = np.random.uniform(0, 3*60)
            time.sleep(lag)

            #Save the html of the listing link. 
            htmlink = browser2.page_source

            #Use bs4 to parse the html response of the listing link
            souplink = bs(htmlink, "html.parser")

            #Type of property
            try: 
                typeProperty = souplink.find_all('span', class_="ds-body ds-home-fact-value")[0].text
            except ValueError as e:
                print(f"{e}Error at type property at {link}")
            
            #Year
            try:
                yearBuilt = int(souplink.find_all('span', class_="ds-body ds-home-fact-value")[1].text)
            except ValueError as e:
                if (souplink.find_all('span', class_="ds-body ds-home-fact-value")[1].text == 'No Data'):
                    continue
                print(f"{e}Error at yearbuilt at {link}")

            #Parking - listed as spaces, space, detached garage, off street, carport
            try:
                try:
                    parking = int(souplink.find_all('span', class_="ds-body ds-home-fact-value")[4].text.replace(" spaces",""))
                except ValueError:
                    pass
                try:
                    parking = int(souplink.find_all('span', class_="ds-body ds-home-fact-value")[4].text.replace(" space",""))
                except ValueError:
                    pass
                try:
                    parking = int(souplink.find_all('span', class_="ds-body ds-home-fact-value")[4].text.replace("Detached Garage","1"))
                except ValueError:
                    pass
                try:
                    parking = int(souplink.find_all('span', class_="ds-body ds-home-fact-value")[4].text.replace("Off street","1"))
                except ValueError:
                    pass
                try:
                    parking = int(souplink.find_all('span', class_="ds-body ds-home-fact-value")[4].text.replace("Carport","1"))
                except ValueError:
                    pass
            except ValueError as e:
                print(f"{e} garage error not determined at index {count}")
    
            #Lot - if the house has HOW the listing will move. listed as sqft and acres.
            try:
                try: 
                    lot = int(souplink.find_all('span', class_="ds-body ds-home-fact-value")[5].text.replace(" sqft","").replace(",",""))
                except ValueError:
                    pass 
                try:
                    lot = int(souplink.find_all('span', class_="ds-body ds-home-fact-value")[6].text.replace(" sqft","").replace(",",""))
                except ValueError:
                    pass
                try:
                    lot = int(souplink.find_all('span', class_="ds-body ds-home-fact-value")[5].text.replace(" acres","").replace(",","")) * 43560
                except ValueError:
                    pass
                try:
                    lot = int(souplink.find_all('span', class_="ds-body ds-home-fact-value")[6].text.replace(" acres","").replace(",","")) * 43560
                except ValueError:
                    pass

            except (ValueError, IndexError) as e:
                print(f"{e} error at lot type at {link}")

            #Sale history
            saleHistory = []
            for i in souplink.find_all('td', class_="kaq65t-1 ijBExC"):
                try:
                    saleHistory.append(i.find('span').text)
                except AttributeError:
                    continue 

            #tax history
            taxHistory = []
            for i in souplink.find_all('td', class_="kaq65t-1 eegcAA"):
                try:
                    taxHistory.append(i.find('span').text)
                except AttributeError:
                    continue

            #Only appends listings with complete information
            if (price and address and rooms and bathrooms and area and link and typeProperty and yearBuilt and parking and lot and saleHistory and taxHistory):

                listingInformation = [price, address, rooms, bathrooms, area, link, typeProperty, yearBuilt, parking, lot, saleHistory, taxHistory]

                #For loop to append the scraped information.
                for key,info in zip(listings_dict.keys(), listingInformation):
                    listings_dict[key].append(info)

            #Adds 1 to the count at the end of the listing scrape
            count += 1
            
        except ValueError as e:
            print(f"{e} error at index {count} page {page}")
            count += 1

    #Adds 1 to the page counter
    page += 1

    #Random lag generator - try to keep the code to be detected as a bot 
    lag = np.random.uniform(0, 1.5*60)
    time.sleep(lag)

    #Finds elements to go next page
    nextpage = browser.find_element_by_css_selector("a.Button-wpcbcc-0.lcWnHB.PaginationButton-si2hz6-0.cUjspl")

    #Zillow only loads 20 page (500 listings) at  time.
    if page <= 20:
        #clicks next page.
        nextpage.click()

    else:
        #When the code is done, this code kills the driver.
        browser.quit()
        browser2.quit()
        print("Scape Done!!!!")
        break

Current page: 1


KeyboardInterrupt: 